In [0]:
import codecs
import numpy as np
import os
import time

import tensorflow as tf
tf.enable_eager_execution()

## Get the Shakespeare dataset

In [0]:
data_fpath = tf.keras.utils.get_file(
    'shakespeare.txt', 
    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

text = codecs.open(data_fpath, 'r', encoding='utf8').read()

print('Length of the text: {} characters'.format(len(text)))
print('=====================================')
print(text[:250])

In [0]:
## Extract the vocabulary

In [0]:
vocab = sorted(set(text))
VOCAB_SIZE = len(vocab)

print ('{} unique characters'.format(VOCAB_SIZE))

## Create char2idx / idx2char dictionaries and convert the text

In [0]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

for char, _ in zip(char2idx, range(20)):
    print('{:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('...')
print('=====================================')
print('Example of the encoded text: {}'.format(text_as_int[:13]))

## Prepare TF data pipeline

In [0]:
SEQ_LEN = 100
BATCH_SIZE = 64

examples_per_epoch = len(text) // SEQ_LEN
steps_per_epoch = examples_per_epoch // BATCH_SIZE

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

dataset = dataset.batch(SEQ_LEN+1, drop_remainder=True)
dataset = dataset.map(split_input_target)

dataset = dataset.shuffle(buffer_size=10000)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

## Take an example from the dataset

In [0]:
for input_example, target_example in dataset.take(1):
    print('=====================================')
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()[1]])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()[1]])))

## Build the model

In [0]:
def build_model(batch_size):
    return tf.keras.Sequential([
        tf.keras.layers.Embedding(VOCAB_SIZE, 256, batch_input_shape=[batch_size, None]),
        tf.keras.layers.CuDNNGRU(1024,
                                 return_sequences=True,
                                 recurrent_initializer='glorot_uniform',
                                 stateful=True),
        tf.keras.layers.Dense(VOCAB_SIZE),
    ])

model = build_model(batch_size=BATCH_SIZE)

model.summary()

![A drawing of the data passing through the model](https://tensorflow.org/tutorials/sequences/images/text_generation_training.png)

## Try the model before training

In [0]:
input_example_batch, target_example_batch = list(dataset.take(1))[0]
example_batch_predictions = model(input_example_batch)

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

print("(batch_size, sequence_length, vocab_size) : {}".format(example_batch_predictions.shape))
print('=====================================')
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print('=====================================')
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

## Define loss and prepare for the training

In [0]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer=tf.train.AdamOptimizer(),
              loss=loss)

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

## Run the training procedure

In [0]:
EPOCHS = 10

history = model.fit(dataset.repeat(), 
                    epochs=EPOCHS, 
                    steps_per_epoch=steps_per_epoch, 
                    callbacks=[checkpoint_callback])

## Restore the latest checkpoint and rebuild the model

In [0]:
model = build_model(batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.summary()

In [0]:
## Define a function for the text generation

In [0]:
def generate_text(model, start_string, num_generate):

    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    temperature = 1.0

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        # Using a multinomial distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

![To generate text the model's output is fed back to the input](https://tensorflow.org/tutorials/sequences/images/text_generation_sampling.png)

## Generate the text

In [0]:
print(generate_text(model, start_string=u"ROMEO:", num_generate=1000))